# Data Preparation

This notebook goes through the tasks necessary to prepare the data for modelling. In particular, I merge the competition provided csv files with the csv files containing additional routing features, filter out some bad rows, create additional features, and split the data into dev/test/train subsets.

## Loading

I use `dask` to load in the csv files because of their size as well as `dask`'s ability to read directly from Google Cloud Storage.

In [1]:
!pip install gcsfs

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split

In [3]:
# specify dtypes of columns to load to speed up loading
dtypes = {
  'train': {
    'fare_amount': 'float32',
    'pickup_datetime': 'str', 
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'uint8',
    'distance': 'float32',
    'duration': 'float32',
    'summary': 'string'
  },
  'test': {
    'key': 'str',
    'pickup_datetime': 'str', 
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'uint8',
    'distance': 'float32',
    'duration': 'float32',
    'summary': 'string'
  }
}

In [4]:
%%time
train = dd.read_csv(
  'gs://nyc-taxi-fare-prediction-data/train_complete.csv',
  usecols=dtypes['train'].keys(), dtype=dtypes['train']).compute()
test = dd.read_csv(
  'gs://nyc-taxi-fare-prediction-data/test_complete.csv',
  usecols=dtypes['test'].keys(), dtype=dtypes['test']).compute()

CPU times: user 5min 39s, sys: 2min 13s, total: 7min 52s
Wall time: 3min 50s


## Missing Data

Before doing anything else, I'll resolve any missing data issues. It appears that there are several rows where the pickup/dropoff point is the same, and then a few where the pickup/dropoff is in the middle of the water.

For the points where the pickup/dropoff is the same, I'll add a boolean feature to indicate this. For the other points where routing info was missing, I'll remove these from the training set altogether. In the test set, there are only two, so I'll simply fill distance with haversine distance and duration with the estimated distance divided by the average speed in the training set.

In [5]:
def haversine(lat1, lon1, lat2, lon2):
  lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
  c = 2 * np.arcsin(np.sqrt(a)) 
  r = 6.371 * 10**6
  return c * r

def add_same_pickup_dropoff_feature(df):
  df['same_pickup_dropoff'] = ((df.pickup_latitude == df.dropoff_latitude) & \
                               (df.pickup_longitude == df.dropoff_longitude))
  df.loc[df.same_pickup_dropoff, 'distance'] = 0
  df.loc[df.same_pickup_dropoff, 'duration'] = 0

def autofill_remaining_nulls(df, avg_speed=10):
  row_slice = ((df.distance == 0) | (df.distance.isnull())) & (~df.same_pickup_dropoff)
  df.loc[row_slice, 'distance'] = haversine(
    df.loc[row_slice, 'pickup_latitude'], df.loc[row_slice, 'dropoff_latitude'],
    df.loc[row_slice, 'pickup_longitude'], df.loc[row_slice, 'dropoff_longitude'])
  df.loc[row_slice, 'duration'] = df.loc[row_slice, 'distance'] / avg_speed

add_same_pickup_dropoff_feature(test)
add_same_pickup_dropoff_feature(train)

test.summary.fillna('', inplace=True)
train.summary.fillna('', inplace=True)

autofill_remaining_nulls(test, avg_speed=np.mean(train.distance / train.duration))
train.dropna(inplace=True) # drop any remaining rows with nulls
train = train[(train.distance != 0) | train.same_pickup_dropoff]  # drop any remaining rows with zeros

print train.isnull().sum()
print test.isnull().sum()

fare_amount            0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude      0
dropoff_latitude       0
passenger_count        0
distance               0
duration               0
summary                0
same_pickup_dropoff    0
dtype: int64
key                    0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude      0
dropoff_latitude       0
passenger_count        0
distance               0
duration               0
summary                0
same_pickup_dropoff    0
dtype: int64


## Outlier Removal

The fare amount is sometimes negative, and there are cases where the lat and lon seem malformed. I'll remove these.

In [6]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,duration
count,5.537930e+07,5.537930e+07,5.537930e+07,5.537930e+07,5.537930e+07,5.537930e+07,5.537930e+07,5.537930e+07
mean,1.134232e+01,-7.255982e+01,3.997507e+01,-7.256441e+01,3.997731e+01,1.685086e+00,5.485846e+03,4.866183e+02
std,2.071259e+01,1.084584e+01,6.682907e+00,1.082842e+01,6.676903e+00,1.327002e+00,2.475589e+04,2.349822e+03
min,-3.000000e+02,-3.440697e+03,-3.488080e+03,-3.440697e+03,-3.488080e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073498e+01,-7.399140e+01,4.073408e+01,1.000000e+00,1.651100e+03,1.812000e+02
50%,8.500000e+00,-7.398181e+01,4.075267e+01,-7.398016e+01,4.075318e+01,1.000000e+00,2.793000e+03,2.953000e+02
75%,1.250000e+01,-7.396712e+01,4.076714e+01,-7.396372e+01,4.076811e+01,2.000000e+00,5.119100e+03,4.959000e+02
max,9.396336e+04,3.456223e+03,3.378013e+03,3.456223e+03,3.378013e+03,2.080000e+02,1.662670e+06,8.521280e+04


In [7]:
test.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,duration
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9.914000e+03,9914.000000
mean,-73.974716,40.751041,-73.973656,40.751740,1.671273,7.672588e+03,399.791199
std,0.042774,0.033541,0.039072,0.035435,1.278747,2.141809e+05,318.346069
min,-74.252190,40.573143,-74.263245,40.568974,1.000000,0.000000e+00,0.000000
25%,-73.992500,40.736125,-73.991249,40.735253,1.000000,1.758275e+03,191.699997
50%,-73.982327,40.753052,-73.980015,40.754065,1.000000,2.927200e+03,307.300003
75%,-73.968012,40.767113,-73.964062,40.768757,2.000000,5.365300e+03,515.475006
max,-72.986534,41.709557,-72.990967,41.696682,6.000000,1.509079e+07,6030.700195


In [8]:
train = train[(train.fare_amount > 0) & (train.fare_amount < 500) & \
              (train.pickup_latitude > 40.5) & (train.pickup_latitude < 41.75) & \
              (train.pickup_longitude > -74.3) & (train.pickup_longitude < -72.96) & \
              (train.dropoff_latitude > 40.5) & (train.dropoff_latitude < 41.75) & \
              (train.dropoff_longitude > -74.3) & (train.dropoff_longitude < -72.96)]

## Splitting

Next, I'll split the training set into train/dev sets. I do this before feature engineering since I'll be using certain statistics from the training set to create features.

In [9]:
np.random.seed(5452)
train, dev = train_test_split(train, test_size=.01)

## Feature Engineering

Next, I create some new features. In particular:

1. Derive year, month, day of week, and hour from the timestamp.
2. Bucket the pickup/dropoff coordinates into percentile buckets.
3. Add distance to/from JFK airport to capture fixed fare rides to Manhattan.
4. Split the summary into two columns.
5. Add bearing.

In [10]:
%%time
def add_datetime_features(df):
  df.pickup_datetime = pd.to_datetime(
    df.pickup_datetime, utc=True, infer_datetime_format=True) \
    .dt.tz_convert('US/Eastern')
  df['year'] = df.pickup_datetime.dt.year
  df['month'] = df.pickup_datetime.dt.month
  df['dow'] = df.pickup_datetime.dt.dayofweek
  df['hour'] = df.pickup_datetime.dt.hour

add_datetime_features(test)
add_datetime_features(train)
add_datetime_features(dev)

CPU times: user 9min 25s, sys: 8.3 s, total: 9min 34s
Wall time: 9min 34s


In [11]:
latitude_quantiles = (pd.concat([train.pickup_latitude, train.dropoff_latitude])
                      .quantile(np.arange(0, 1.1, .1)))
longitude_quantiles = (pd.concat([train.pickup_longitude, train.dropoff_longitude])
                       .quantile(np.arange(0, 1.1, .1)))
labels = map(str, range(latitude_quantiles.size - 1))

In [12]:
def add_coordinate_quantiles(df):
  df['pickup_latitude_quantile'] = pd.cut(
    df.pickup_latitude, latitude_quantiles, right=True, include_lowest=True, labels=labels)
  df['dropoff_latitude_quantile'] = pd.cut(
    df.dropoff_latitude, latitude_quantiles, right=True, include_lowest=True, labels=labels)
  df['pickup_longitude_quantile'] = pd.cut(
    df.pickup_longitude, longitude_quantiles, right=True, include_lowest=True, labels=labels)
  df['dropoff_longitude_quantile'] = pd.cut(
    df.dropoff_longitude, longitude_quantiles, right=True, include_lowest=True, labels=labels)
  
add_coordinate_quantiles(test)
add_coordinate_quantiles(train)
add_coordinate_quantiles(dev)

In [13]:
def add_airport_features(df):
  jfk = (40.644166, -73.782222)
  df['pickup_distance_to_jfk'] = haversine(
    jfk[0], jfk[1], df['pickup_latitude'], df['pickup_longitude'])
  df['pickup_jfk'] = (df.pickup_distance_to_jfk < 1500).astype('int')
  df['dropoff_distance_to_jfk'] = haversine(
    jfk[0], jfk[1], df['dropoff_latitude'], df['dropoff_longitude'])
  df['dropoff_jfk'] = (df.dropoff_distance_to_jfk < 1500).astype('int')

add_airport_features(test)
add_airport_features(train)
add_airport_features(dev)

In [14]:
%%time
def add_summary_features(df):
  df['summary1'], df['summary2'] = df.summary.str.split(', ', 1).str
  df.summary1.fillna('', inplace=True)
  df.summary2.fillna('', inplace=True)

add_summary_features(train)
add_summary_features(test)
add_summary_features(dev)

CPU times: user 3min 25s, sys: 13 s, total: 3min 38s
Wall time: 3min 37s


In [15]:
%%time
def force_vocabulary_for_summaries(df, s1_vocab, s2_vocab):
  df['summary1'] = df.summary1.map(lambda x: 'Other' if x not in s1_vocab else x)
  df['summary2'] = df.summary2.map(lambda x: 'Other' if x not in s2_vocab else x)

s1_counts = train.summary1.value_counts()
s2_counts = train.summary2.value_counts()
s1_vocab = set(s1_counts[s1_counts / train.shape[0] > .002].index)
s2_vocab = set(s2_counts[s2_counts / train.shape[0] > .002].index)

force_vocabulary_for_summaries(train, s1_vocab, s2_vocab)
force_vocabulary_for_summaries(test, s1_vocab, s2_vocab)
force_vocabulary_for_summaries(dev, s1_vocab, s2_vocab)

CPU times: user 1min 3s, sys: 0 ns, total: 1min 3s
Wall time: 1min 3s


In [32]:
def bearing(lat1, lon1, lat2, lon2, bucket=False):
  lat1 = np.radians(lat1)
  lat2 = np.radians(lat2)
  dlon = np.radians(lon2 - lon1)
  x = np.sin(dlon) * np.cos(lat2)
  y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1)
          * np.cos(lat2) * np.cos(dlon))
  initial_bearing = np.degrees(np.arctan2(x, y))
  compass_bearing = (initial_bearing + 360) % 360
  if bucket:
    bucket_bearing = pd.cut(compass_bearing, bins=16, labels=range(1, 16) + [0])
    bucket_labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    bucket_lookup = {i: bucket_labels[i/2] for i in range(16)}
    return bucket_bearing.apply(lambda x: bucket_lookup[x])
  else:
    return compass_bearing

def add_bearing_features(df):
  df['bearing_degrees'] = bearing(
    df.pickup_latitude, df.pickup_longitude, df.dropoff_latitude, df.dropoff_longitude)
  df['bearing_bucket'] = bearing(
    df.pickup_latitude, df.pickup_longitude, df.dropoff_latitude, df.dropoff_longitude, bucket=True)
  df.loc[same_pickup_dropoff, 'bearing_bucket'] = 'None'
  
add_bearing_features(test)
add_bearing_features(train)
add_bearing_features(dev)

In [17]:
test['same_pickup_dropoff'] = test.same_pickup_dropoff.astype('int')
train['same_pickup_dropoff'] = train.same_pickup_dropoff.astype('int')
dev['same_pickup_dropoff'] = dev.same_pickup_dropoff.astype('int')

## Save

Finally, I'll save the final files to be easily loadable for training.

In [36]:
test.to_csv('data/test.csv', index=False)
train.to_csv('data/train.csv', index=False)
dev.to_csv('data/dev.csv', index=False)

In [19]:
# These define the vocab for the summary1/2 columns and can be loaded to
# tensorboard for visualization.
pd.Series(sorted(list(s1_vocab))).to_csv('data/s1_vocab.csv', index=False, header=False)
pd.Series(sorted(list(s2_vocab))).to_csv('data/s2_vocab.csv', index=False, header=False)

In [20]:
train.columns

Index([u'fare_amount', u'pickup_datetime', u'pickup_longitude',
       u'pickup_latitude', u'dropoff_longitude', u'dropoff_latitude',
       u'passenger_count', u'distance', u'duration', u'summary',
       u'same_pickup_dropoff', u'year', u'month', u'dow', u'hour',
       u'pickup_latitude_quantile', u'dropoff_latitude_quantile',
       u'pickup_longitude_quantile', u'dropoff_longitude_quantile',
       u'pickup_distance_to_jfk', u'pickup_jfk', u'dropoff_distance_to_jfk',
       u'dropoff_jfk', u'summary1', u'summary2', u'bearing_degrees',
       u'bearing_bucket'],
      dtype='object')

In [21]:
test.columns

Index([u'key', u'pickup_datetime', u'pickup_longitude', u'pickup_latitude',
       u'dropoff_longitude', u'dropoff_latitude', u'passenger_count',
       u'distance', u'duration', u'summary', u'same_pickup_dropoff', u'year',
       u'month', u'dow', u'hour', u'pickup_latitude_quantile',
       u'dropoff_latitude_quantile', u'pickup_longitude_quantile',
       u'dropoff_longitude_quantile', u'pickup_distance_to_jfk', u'pickup_jfk',
       u'dropoff_distance_to_jfk', u'dropoff_jfk', u'summary1', u'summary2',
       u'bearing_degrees', u'bearing_bucket'],
      dtype='object')

In [22]:
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,duration,same_pickup_dropoff,year,month,dow,hour,pickup_distance_to_jfk,pickup_jfk,dropoff_distance_to_jfk,dropoff_jfk,bearing_degrees
count,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07,5.369053e+07
mean,1.131340e+01,-7.397073e+01,4.075014e+01,-7.397050e+01,4.075024e+01,1.685298e+00,4.488682e+03,3.885669e+02,1.074672e-02,2.011736e+03,6.272044e+00,2.984727e+00,1.182210e+01,2.040605e+04,1.475249e-02,2.045714e+04,7.007102e-03,1.522104e+02
std,9.617471e+00,3.830372e-02,2.947073e-02,3.744699e-02,3.272661e-02,1.315083e+00,5.043731e+03,3.063952e+02,1.031078e-01,1.862776e+00,3.437314e+00,1.905277e+00,5.835394e+00,3.254866e+03,1.205606e-01,3.025989e+03,8.341464e-02,1.045948e+02
min,1.000000e-02,-7.429988e+01,4.050004e+01,-7.429984e+01,4.050003e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.008000e+03,1.000000e+00,0.000000e+00,0.000000e+00,6.901397e-01,0.000000e+00,6.901397e-01,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399228e+01,4.073658e+01,-7.399158e+01,4.073560e+01,1.000000e+00,1.703200e+03,1.867000e+02,0.000000e+00,2.010000e+03,3.000000e+00,1.000000e+00,7.000000e+00,2.010701e+04,0.000000e+00,2.007159e+04,0.000000e+00,4.233578e+01
50%,8.500000e+00,-7.398209e+01,4.075338e+01,-7.398060e+01,4.075387e+01,1.000000e+00,2.838700e+03,2.997000e+02,0.000000e+00,2.012000e+03,6.000000e+00,3.000000e+00,1.200000e+01,2.072996e+04,0.000000e+00,2.072423e+04,0.000000e+00,1.692348e+02
75%,1.250000e+01,-7.396831e+01,4.076756e+01,-7.396535e+01,4.076840e+01,2.000000e+00,5.165700e+03,4.993000e+02,0.000000e+00,2.013000e+03,9.000000e+00,5.000000e+00,1.700000e+01,2.142290e+04,0.000000e+00,2.144643e+04,0.000000e+00,2.227244e+02
max,4.999900e+02,-7.296081e+01,4.174669e+01,-7.296043e+01,4.174704e+01,2.080000e+02,2.145980e+05,1.036680e+04,1.000000e+00,2.015000e+03,1.200000e+01,6.000000e+00,2.300000e+01,1.364345e+05,1.000000e+00,1.364263e+05,1.000000e+00,3.599985e+02


In [23]:
test.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,duration,same_pickup_dropoff,year,month,dow,hour,pickup_distance_to_jfk,pickup_jfk,dropoff_distance_to_jfk,dropoff_jfk,bearing_degrees
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9.914000e+03,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974716,40.751041,-73.973656,40.751740,1.671273,7.672588e+03,399.791199,0.008574,2011.815312,6.861307,2.842243,12.404983,20417.677734,0.015836,20469.933594,0.005749,151.149506
std,0.042774,0.033541,0.039072,0.035435,1.278747,2.141809e+05,318.346069,0.092201,1.803435,3.353546,1.937365,6.031902,3819.835693,0.124848,3298.526855,0.075611,104.615349
min,-74.252190,40.573143,-74.263245,40.568974,1.000000,0.000000e+00,0.000000,0.000000,2009.000000,1.000000,0.000000,0.000000,11.431990,0.000000,9.966619,0.000000,0.000000
25%,-73.992500,40.736125,-73.991249,40.735253,1.000000,1.758275e+03,191.699997,0.000000,2010.000000,4.000000,1.000000,8.000000,20109.707031,0.000000,20065.920898,0.000000,41.429848
50%,-73.982327,40.753052,-73.980015,40.754065,1.000000,2.927200e+03,307.300003,0.000000,2012.000000,7.000000,3.000000,13.000000,20736.502930,0.000000,20728.685547,0.000000,165.850464
75%,-73.968012,40.767113,-73.964062,40.768757,2.000000,5.365300e+03,515.475006,0.000000,2014.000000,10.000000,5.000000,17.000000,21421.462891,0.000000,21456.934570,0.000000,220.582008
max,-72.986534,41.709557,-72.990967,41.696682,6.000000,1.509079e+07,6030.700195,1.000000,2015.000000,12.000000,6.000000,23.000000,135899.171875,1.000000,134473.171875,1.000000,359.926239
